# **Exploratory Data Analisis** (EDA)
En este notebook me enfoco en:
- Recolección de datos desde distintas fuentes
- Identificado diversas estructuras de los datasets
- Análisis de nulos, duplicados, relaciones
- Checklist de problemas identificados

## 1. Configuración inicial
- Importación de librerias

In [ ]:

import sys, os

# Obtengo la ruta raíz del proyecto (subo un nivel desde /notebooks)
project_root = os.path.abspath("../")

# La agrego al path si no está
if project_root not in sys.path:
    sys.path.append(project_root)

# Importado de librerías
from scripts.utils import realizar_request, detectar_outliers_iqr, pasar_snake_case
import pandas as pd
import os
import json
import subprocess
import tabula

## 2. Obtención de datasets

- Obtención del dataset de ventas y paso a DataFrame

In [ ]:
# Cargar credenciales de Kaggle desde un archivo JSON
with open(r"..\kaggle_keys.json") as f:
    kaggle_creds = json.load(f)

os.environ["KAGGLE_USERNAME"] = kaggle_creds["username"]
os.environ["KAGGLE_KEY"] = kaggle_creds["key"]

dataset = "virtualschool/restaurant-cost-and-sales-dataset"

# Ejecuta el comando kaggle datasets download
subprocess.run([
    "kaggle", "datasets", "download",
    "-d", dataset,
    "--unzip",
    "-p", '../data/raw'
])

dir_raw = r"..\data\raw"
excel_restaurante = dir_raw + r"\Restaurant_Data.xlsx"

df_orders = pd.read_excel(excel_restaurante, sheet_name="Orders")
df_items = pd.read_excel(excel_restaurante, sheet_name="Items")

- Dataset del INDEC

In [ ]:
path_informe = dir_raw + r"\INFORME_INDEC_1-10-25.pdf"

# Obtener DataFrame que me interesa
df_indec = tabula.read_pdf(
    path_informe,
    pages=12,
    multiple_tables=True,
    encoding='latin1'  
)[0]

- Valor del dólar

In [ ]:
url_dolarapi = 'https://dolarapi.com/v1/dolares/blue'

request_ok, data = realizar_request(url_dolarapi)
if not request_ok:
    raise RuntimeError("No se pudo obtener el valor del dólar desde la API.")
dolar_blue = float(data.get("venta"))

## 3. Analizado de DataFrames y detección de tareas
- Vista general

In [ ]:
print(df_orders.info())
print('------------------')
print(df_items.info())
print('------------------')
print(df_indec.info())

> Dado a lo que pareciera ser un problema en las columnas del INDEC, realizo una vista más detallada

In [ ]:
print(df_indec.head())

> Los nombres de las columnas parecieran no estar perfectamente alineados con el encabezado del DataFrame, pero están presentes!

- Ahora, **detección de nulos y duplicados**:

In [ ]:
dfs = {
    "df_orders": df_orders,
    "df_items": df_items,
    "df_indec": df_indec
}

for nombre, df in dfs.items():
    print(f"\n-> {nombre} - {df.isnull().sum().sum()} valores nulos --- {df.duplicated().sum().sum()} valores duplicados")

- Búsqueda de outliers

In [ ]:
# DataFrame rápido para la detección de
indec_outliers = pd.DataFrame()
indec_outliers = df_indec
indec_outliers["precio"] = indec_outliers.iloc[5:, 3].dropna().map(pasar_snake_case).astype(float)

outliers_indec = detectar_outliers_iqr(indec_outliers, 'precio')
print(f'-> df_indec - Columna precio: {len(outliers_indec)}\n')

outliers_precio = detectar_outliers_iqr(df_items, 'Price')
outliers_costo = detectar_outliers_iqr(df_items, 'Cost')
print(f'-> df_indec - Columna Price: {len(outliers_precio)} - Columna Cost: {len(outliers_costo)}\n')

outliers_count = detectar_outliers_iqr(df_orders, 'Count')
print(f'-> df_indec - Columna Count: {len(outliers_precio)}\n')

> **Nota:** Las únicas columnas que me interesan son aquellas numéricas que no contienen FKs

- Dado a los outliers en una tabla de dimensiones, intento analizarlos más de cerca

In [ ]:
print(outliers_indec)

En este caso en particular, los outliers son razonables y concuerdan con los productos

## 4. Pasado a formato CSV para la posterior realización de tareas

In [ ]:
staging_dir = r"..\data\staging"

# Almacenado de los DataFrames en formato CSV
df_orders.to_csv(staging_dir + r"\orders.csv", index=False)
df_items.to_csv(staging_dir + r"\items.csv", index=False)
df_indec.to_csv(staging_dir + r"\indec.csv", index=False)

# Guardo el valor del dólar blue en un archivo de texto
with open(staging_dir + r"\dolar_blue.txt", "w") as f:
    f.write(str(dolar_blue))


## 5. Síntesis del Exploratory Data Analysis

**Se puede apreciar fácilmente:**
- Diferencias de lenguaje: INDEC en español, pero ventas en inglés
- Diferencias entre formatos (preferible pasarlo todo a `snake_case`)
- Valores nulos varios: 77 en el INDEC y 6 en Items
- Formateo necesario del DataFrame del INDEC con asignación de columnas
- Relación entre Items (dimensión) y Orders (factos)

**A realizar:**
- Asignación de columnas que me interesan del INDEC (`Variedad`, `Unidad de medida` y `Octubre de 2025`)
- Limpieza de valores nulos
- Traducción al español del dataset de Kaggle
- Formateo a `snake_case` y eliminación de tildes
- Pasado de USD a ARS
- Generalización de unidades de medida a kilos.
